In [1]:
# %load project_utils.py
import sys
from os import environ
from secret import sql_password, spotify_credentials
import pandas as pd
import numpy as np
import psycopg2 as pg
from psycopg2 import Error
from spotipy.oauth2 import SpotifyClientCredentials
import librosa
import spotipy
import requests
from genre_replace import genre_replace
from sklearn.metrics.pairwise import cosine_similarity

client_credentials_manager = SpotifyClientCredentials(client_id=spotify_credentials['client_id'],
                                                      client_secret=spotify_credentials['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#=============================== SQL Utils ====================================#
conn = pg.connect(database="spotify_db",
                  user="postgres", 
                  password=sql_password)

def run_query(q):
    '''a function that takes a SQL query as an argument
    and returns a pandas dataframe of that query'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(q)
            return pd.read_sql(q, conn)

        except (Exception, pg.DatabaseError) as error:
            print(error)

#============================= Spotify Utils ==================================#
def search_and_extract(track_query):
    '''A function that takes in a song query and returns
    the track id and preview url for that track in a dict.'''

    track_query = str(track_query)

    #uses the API to search for a track
    search = sp.search(track_query, type='track', limit=1, market='US')

    track_id = search['tracks']['items'][0]['id']
    preview_url = search['tracks']['items'][0]['preview_url']
    track_name = search['tracks']['items'][0]['name']
    artist = search['tracks']['items'][0]['artists'][0]['name']
    artist_id = search['tracks']['items'][0]['artists'][0]['id']

    return track_id, preview_url, track_name, artist, artist_id

def get_artist_genre(artist_id):
    '''A function that takes in a Spotify artist id, calls the Spotify 
    API, and returns the artist genres, as a list'''
    search = sp.artist(artist_id)
    return search['genres']
    
def extract_features(track_id):
    '''A function that takes in a spotify track id, requests the audio
    features using the 'audio_features' endpoint from the Spotify API,
    and returns the features as a dataframe'''
    track_id = str(track_id)
    features = sp.audio_features(track_id)
	#     features[0].values()

    spotify_features = pd.DataFrame(data=features[0].values(),index=features[0].keys())
    spotify_features = spotify_features.transpose()
    spotify_features.drop(['type','uri','track_href','analysis_url'],axis=1,inplace=True)

    return spotify_features

#============================= Librosa Utils ==================================#
def check_for_track_preview(url):
    '''Given a url object, checks if the track has a
        preview'''
    if url == None:
        return False
    else:
        return True

def get_mp3(url,track_id):
    '''A function that takes an mp3 url, and writes it to the local
        directory "audio-files"'''
    try:
        doc = requests.get(url)
        with open(f'audio-files/track_{track_id}.mp3', 'wb') as f:
            f.write(doc.content)
    except:
        pass

def librosa_pipeline(track_id):
    '''This function takes in a spotify track_id as a string
        and uploads the cooresponding mp3 preview from a local
        directory. The mp3 then goes through the feature
        extraction process. A dictionary is returned with each
        audio feature as a key and their cooresponding value.

        REQUIREMENTS:
        * MP3 file must be in the directory in the form below
        '''

    track = f'audio-files/track_{track_id}.mp3'

    d = {}
    d['track_id'] = track_id

    #load mp3
    y, sr = librosa.load(track, mono=True, duration=30)

    #feature extraction
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    d['spectral_centroid'] = np.mean(spec_cent)

    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    d['spectral_bandwidth'] = np.mean(spec_bw)

    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    d['rolloff'] = np.mean(rolloff)

    zcr = librosa.feature.zero_crossing_rate(y)
    d['zero_crossing_rate'] = np.mean(zcr)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for i,e in zip(range(1, 21),mfcc):
            d[f'mfcc{i}'] = np.mean(e)

    chroma = ['C', 'C#', 'D','D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    for c,p in zip(chroma,chroma_stft):
        d[c] = np.mean(p)

    return d    

#============================= General Utils ==================================#
def check_query_format(query):
    query = query[:-1] if query.endswith(';') else query
    query = query.split(";")

    for track in query:
        track = track.split(",")
        try:
            name = track[0].strip()
            artist = track[1].strip()
        except IndexError:
            return False

def sort_inputs(query):
    not_in_db = []
    in_db = []
    query = query.replace("'","_")
    query = query[:-1] if query.endswith(';') else query
    query = query.split(";")
    
    for track in query:
        track = track.split(",")

        name = track[0].strip()
        artist = track[1].strip()

        q = f'''SELECT * FROM norm_tracks
        WHERE track_name ILIKE '%{name}%'
        AND artist ILIKE '%{artist}%';
        '''
        r = run_query(q)
        
        name = name.replace("_","'")
        
        if len(r) > 0:
            in_db.append(name + "," + artist)
        else:
            not_in_db.append(name + " " + artist)

    return [in_db, not_in_db]

def parse_and_sort_inputs(user_a_query, user_b_query):
    '''Takes in both user's input strings, and sets up a 
    dictionary to keep track of each user's inputs and 
    whether they are in the database or not. Calls the 
    sort_inputs function to parse and sort query strings.
    Returns the resulting dictionary'''
    # example user inputs
    # user_a = "malibu, miley cyrus; video games, lana del rey; you're no good, linda ronstadt"
    # user_b = "don't stop me now, queen; rocket man, elton john; toxic, britney spears"

    # combines the form input into a list for interation; dict to store tracks
    users = [user_a_query, user_b_query]
    initial_inputs = {'user_a':None,
                    'user_b':None}

    # for each set of tracks, I need to keep track which tracks are in/not in the DB
    for key,user in zip(initial_inputs.keys(),users):   
        in_db, not_in_db = sort_inputs(user)
        initial_inputs[key] = [in_db, not_in_db]


    return initial_inputs

def cos_sim(a,b):
    '''Calculates the cosine similarity between two feature
        vectors'''
    d = np.dot(a, b)
    l = (np.linalg.norm(a))*(np.linalg.norm(b))
    return d/l


# # for creating a spotify playlist from track_uris
# def create_playlist(sp, recommended_tracks):
#     user_all_data = sp.current_user()
#     user_id = user_all_data["id"]

#     playlist_all_data = sp.user_playlist_create(user_id, "Friendship Playlist")
#     playlist_id = playlist_all_data["id"]
#     playlist_uri = playlist_all_data["uri"]
#     # try:
#     sp.user_playlist_add_tracks(user_id, playlist_id, recommended_tracks)
#     # except spotipy.client.SpotifyException as s:
#     # 	print("could not add tracks")

#     return playlist_uri
#================================== IN DATABASE ===============================#
def in_database(in_db):
    '''takes in a list of tracks, parses it,
    queries the db for each track's feature 
    vector and genre, the appends each to
    a df, then returns the df'''

    in_db_df = pd.DataFrame()
    for t in in_db:
        track = t.split(",")
        name = track[0]
        artist = track[1]

        name = name.replace("'","_")

        q = f'''SELECT a.*, b.genre 
            FROM norm_tracks a JOIN tracks b
            ON a.track_id = b.track_id
            WHERE a.track_name ILIKE '%{name}%'
            AND a.artist ILIKE '%{artist}%'
            LIMIT 1;
            '''
        r = run_query(q)
        in_db_df = in_db_df.append(r)
    

    return in_db_df.reset_index(drop=True)
#================================ NOT IN DATABASE =============================#
def not_in_database(not_in_db):
    #search for a track and extract metadata from results
    metadata = {}
    for track in not_in_db:
        track_id, preview_url, track_name, artist, artist_id = search_and_extract(track) #using the input track name as the query to search spotify
        genres = get_artist_genre(artist_id)
        metadata[track_id] = [preview_url,track_name,artist,artist_id,genres]

    not_in_db_df = pd.DataFrame()
    # no_url = {}
    for track_id in metadata.keys():
        if metadata[track_id][0] == None:
            # no_url[track_id] = [metadata[track_id][1],metadata[track_id][2]]
            # continue
            pass
        
        spotify_features = extract_features(track_id)
        get_mp3(metadata[track_id][0],track_id)

        #use librosa to extract audio features
        r = librosa_pipeline(track_id)

        #turning dict into datframe
        librosa_features = pd.DataFrame(r,index=[0])

        #concatenating the two dfs so the feature vector will be in the same format as the db
        all_features = pd.concat([librosa_features,spotify_features],axis=1)
        all_features.drop(['id','duration_ms','time_signature','mode','key'],axis=1, inplace=True)

        #insert metadata into dataframe
        all_features.insert(1,'track_name',metadata[track_id][1])
        all_features.insert(2,'artist',metadata[track_id][2])
        all_features.insert(48,'genre',metadata[track_id][4][0])
        
        not_in_db_df = not_in_db_df.append(all_features)
    
    not_in_db_df = not_in_db_df.reset_index(drop=True)
    return not_in_db_df

def scale_features(not_in_db_df):
    # min-max scaling
    #querying for the database
    q = '''SELECT a.*, b.*
        FROM librosa_features a 
        JOIN spotify_features b ON a.track_id = b.id;'''

    database = run_query(q)
    database.drop(['id','duration_ms','time_signature','mode','key'],axis=1, inplace=True)
    i = len(database)
    fv = not_in_db_df.drop(['track_name','artist','genre'],axis=1)

    #append feature vector to bottom of the db
    database = pd.concat([database.iloc[:,1:],fv.iloc[:,1:]],ignore_index=True)

    # #apply a lambda function that does min-max normalization on the db
    database = database.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    
    #overwrite features vector df
    not_in_db_df.iloc[:,3:-1] = database.iloc[i:,:].values
    return not_in_db_df

def remap_genres(df):
    for i,row in df.iterrows():
        if isinstance(row['genre'],str):
            df.loc[i,'genre'] = genre_replace[row['genre']]

        elif isinstance(row['genre'],list):
            for g in row['genre']:
                if g in genre_replace.keys():
                    df.loc[i,'genre'] = genre_replace[g]
                else:
                    pass
                
    return df


#============================= Combining Steps ================================#
def generate_user_df(user_lists):
    '''MUST BE CALLED ON EACH USER KEY SEPARATELY
    Takes in the keys of the initial_inputs dictionary.
    This function calls the in_database and not_in_database
    functions, then concatenates them to create the final
    user dataframes needed to make recommendations. It
    also stores the songs that could not be analyzed in the
    no_url dictionary'''
    
    in_db_df = in_database(user_lists[0])
    not_in_db_df = not_in_database(user_lists[1])
    
    # if not_in_db_df.empty:
    #     user_df = in_db_df
    # else:
    #     not_in_db_df = scale_features(not_in_db_df)
    #     user_df = pd.concat([in_db_df,not_in_db_df],ignore_index=True)
        
    return in_db_df, not_in_db_df

def get_similar_track_ids(input_track_df):
    '''
    IMPORTANT:THIS FUNCTION IS MEANT FOR ITERATION
    ----------------------------------------------
    Takes in a pandas series of a single track
    that contains track_id, and genre. Then queries
    the db for all tracks in the same genre as the
    input track. The cosine similarity is then 
    calculated between the input track and all
    other tracks within the genre. The top two
    most similar track ids are returned in a list'''
    
    name = input_track_df['track_name'].replace("'","_")

    q2 = f'''
    SELECT a.*, b.genre 
    FROM tracks b
    JOIN norm_tracks a ON b.track_id = a.track_id
    WHERE b.genre = '{input_track_df['genre']}'
    AND a.track_id != '{input_track_df['track_id']}' 
    AND a.track_name NOT LIKE '%{name}%';'''
    genre_tracks = run_query(q2)
    
    
    all_scores = {}
    for i,row in genre_tracks.iterrows():
        track_id = row['track_id']
        score = cos_sim(input_track_df[3:-1],row[3:-1])
        all_scores[track_id] = score

    most_similar = sorted(all_scores, 
                          key=all_scores.get,
                          reverse=True)[:2]
    return most_similar

def get_feature_vector_array(id_list):
    '''
    IMPORTANT:THIS FUNCTION IS MEANT FOR ITERATION
    ----------------------------------------------
    Takes in a list of track_ids, queries the
    db for each track's feature vector, and returns
    a 2D array of the feature vectors and cooresponding
    track_ids as an index.
    '''
    id_list = set(id_list)
    q = f'''
    SELECT * FROM norm_tracks
    WHERE track_id IN {tuple(id_list)};'''
    fv = run_query(q)

    fv = fv.set_index('track_id')
    index = fv.index
    fv = fv.iloc[:,2:]
    array = fv.values
    
    return index, array
#============================== Final Steps ==================================#
def create_similarity_matrix(user_a_array, user_a_index, user_b_array, user_b_index):
    '''Takes in two 2D user arrays and their corresponding 
    track_id indices, calculates the cosine similarity
    between all tracks in each 2D array. Then sets up a
    pandas dataframe of the similarity scores
    '''
    cosine_matrix = cosine_similarity(user_a_array,user_b_array)

    cosine_df = pd.DataFrame(cosine_matrix,
                            columns=user_b_index,
                            index=user_a_index)

    return cosine_df

def get_combined_recommendations(cosine_df):
    '''Takes in the cosine similarity dataframe as an
    input, then finds the pairs of track that have 
    the top 3 similarity scores. Queries the db
    for the track metadata and uses the results as the
    final recommendations'''
    
    scores = {}
    for i,row in cosine_df.iterrows():
        scores[max(row)] = [i,row.idxmax()]
        
    top_three = sorted(scores,reverse=True)[:3]

    ids = [scores[i][0] for i in top_three] + [scores[i][1] for i in top_three]
    ids = set(ids)

    q = f'''
    SELECT track_id, track_name, artist, genre FROM tracks
    WHERE track_id IN {tuple(ids)};'''
    final = run_query(q)
    return final

In [3]:
q = '''SELECT * FROM track_metadata'''
data = run_query(q)

data

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,single,6IpBMtAMrBvSHc9y6rTtQQ,Real Love Baby,189293.0,66.0,https://p.scdn.co/mp3-preview/a73451291399b823...,"['art pop', 'chamber pop', 'freak folk', 'indi...",indie,art pop
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,single,4T3wWtIP6KqYa0KRZiuYiG,In a River,223573.0,58.0,https://p.scdn.co/mp3-preview/a47af0dc9a8a5582...,"['art pop', 'chamber pop', 'indie pop', 'indie...",indie,art pop
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,1BOfOlZo9Nzx7SmYAucY9t,Fear Fun,234160.0,59.0,https://p.scdn.co/mp3-preview/c209b9ccde129e96...,"['art pop', 'chamber pop', 'freak folk', 'indi...",indie,art pop
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,2A8IKX257C4hJaYUHMhLP7,"I Love You, Honeybear",170840.0,58.0,https://p.scdn.co/mp3-preview/1f9ea14fa26f55bb...,"['art pop', 'chamber pop', 'freak folk', 'indi...",indie,art pop
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,album,1BOfOlZo9Nzx7SmYAucY9t,Fear Fun,190613.0,56.0,https://p.scdn.co/mp3-preview/6aceffbff71010d0...,"['art pop', 'chamber pop', 'freak folk', 'indi...",indie,art pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33036,0a4agFmqHXxcZl1nho1BxM,Heart Of Glass - Special Mix,Blondie,4tpUmLEVLCGFr93o8hFFIB,compilation,7mEjsBlRmfP63cH1gdPT6A,Best Of Blondie,275733.0,69.0,None,"['candy pop', 'dance rock', 'new romantic', 'n...",pop,candy pop
33037,1tsWNtWxi7SVydMm2G1W6Z,If I Can't Have You - 2007 Remastered Version ...,Yvonne Elliman,2d6JU9LvNhZR7AAtu4x2rS,album,0taUwU7qjtc9lvwmd7FKac,Saturday Night Fever [The Original Movie Sound...,180586.0,4.0,None,['disco'],disco,disco
33038,3GGcwG519BTMdvMeFy7meT,Lady Marmalade,Patti LaBelle,0ty0xha1dbprYIUAQufkFn,compilation,2fgrmnOGJzwU79xlHe4Iub,Best Of Patti Labelle,235266.0,54.0,https://p.scdn.co/mp3-preview/da137cbab59ef54b...,"['disco', 'funk', 'motown', 'neo soul', 'new j...",disco,disco
33039,1ip1ULxxvAzaqH3VqKObdu,Can't Take My Eyes Off of You - Black Box Mix,Gloria Gaynor,6V6WCgi7waF55bJmylC4H5,album,3mn5uMVqqN1WPA2Jwd8lfk,Gloria Gaynor '90 (All New Versions),316000.0,52.0,https://p.scdn.co/mp3-preview/76fdc127a5fb13ec...,['disco'],disco,disco


In [36]:
results = data.groupby(["track_name", "artist"]).size().sort_values(ascending=False)                               
results[results > 1][:50]

track_name                                              artist                      
We're Not Gonna Take It                                 Twisted Sister                  3
Beautiful Girls                                         Sean Kingston                   3
All Right Now                                           Free                            3
The Joker                                               Steve Miller Band               3
Mayores                                                 Becky G                         2
Wake Up in the Sky                                      Gucci Mane                      2
Green River                                             Creedence Clearwater Revival    2
Chelsea Dagger                                          The Fratellis                   2
Most Girls                                              Hailee Steinfeld                2
When the Night is Over                                  Lord Huron                      2
Maybe in the Su

In [38]:
data[data['artist']=='Lizzo'].sort_values(['track_name','track_preview_url'])

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
1888,1du4tPkvJfCIuVL6WMtqSs,Boys,Lizzo,56oDRnqbIiwx4mymNEv7dS,album,7hBV0wo7cDHZQLYnuOJ312,Cuz I Love You (Super Deluxe),172630.0,64.0,https://p.scdn.co/mp3-preview/a2db1ef85045a886...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
26207,5MczYuYdsF4IE1b8ZEZqfy,Boys - Black Caviar Remix,Lizzo,56oDRnqbIiwx4mymNEv7dS,single,1OhlGc0ktAKbD1GqlfvpcK,Boys (Remixes),234098.0,39.0,https://p.scdn.co/mp3-preview/0a53bb5042bb23c5...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
1886,6YdQgWSpsxhVeX6Xmv3IFJ,Cuz I Love You,Lizzo,56oDRnqbIiwx4mymNEv7dS,album,6dFFcYQ8VhifgdKgYY5LYL,Cuz I Love You,179978.0,72.0,https://p.scdn.co/mp3-preview/04472d81bc5a9669...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
6040,1IOST4IzZK644vqdEFkPzg,Exactly How I Feel (feat. Gucci Mane),Lizzo,56oDRnqbIiwx4mymNEv7dS,album,74gSdSHe71q7urGWMMn3qB,Cuz I Love You (Deluxe),143190.0,53.0,https://p.scdn.co/mp3-preview/eed98a5739b6ab66...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
8059,6uAm7pG66O1XesXS9bpHSF,Good As Hell,Lizzo,56oDRnqbIiwx4mymNEv7dS,single,5PMpdvn38HScK6IuLniRep,Good As Hell,159011.0,23.0,None,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
1884,6KgBpzTuTRPebChN0VTyzV,Good as Hell,Lizzo,56oDRnqbIiwx4mymNEv7dS,album,7hBV0wo7cDHZQLYnuOJ312,Cuz I Love You (Super Deluxe),159735.0,70.0,https://p.scdn.co/mp3-preview/26fb41451edf62e4...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
4907,3Yh9lZcWyKrK9GjbhuS0hR,Good as Hell,Lizzo,56oDRnqbIiwx4mymNEv7dS,single,4CA8GvVF7swZafg0zYh9qq,Good as Hell,159735.0,77.0,https://p.scdn.co/mp3-preview/26fb41451edf62e4...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
1881,2rjjNUYONZqC7wSd1i6OFq,Good as Hell (feat. Ariana Grande),Lizzo,56oDRnqbIiwx4mymNEv7dS,album,7hBV0wo7cDHZQLYnuOJ312,Cuz I Love You (Super Deluxe),159011.0,67.0,https://p.scdn.co/mp3-preview/95a1c1a7a990490f...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
1078,07Oz5StQ7GRoygNLaXs2pd,Good as Hell (feat. Ariana Grande) - Remix,Lizzo,56oDRnqbIiwx4mymNEv7dS,single,1k1HuvFs562Z3CCiSYhtc1,Good as Hell (feat. Ariana Grande) [Remix],159011.0,86.0,https://p.scdn.co/mp3-preview/95a1c1a7a990490f...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room
6035,4l21VaodybkZaNRqwzSBZl,Jerome,Lizzo,56oDRnqbIiwx4mymNEv7dS,album,74gSdSHe71q7urGWMMn3qB,Cuz I Love You (Deluxe),231570.0,56.0,https://p.scdn.co/mp3-preview/8f662de51ad3f613...,"['escape room', 'minnesota hip hop', 'pop', 'p...",hip hop,escape room


In [37]:
data[data['track_name'].str.contains("Remix") == True]
# test = data.groupby(["track_name"]).size().sort_values(ascending=False)
# test

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
13,4RXyvXsnYwPekkN5Ptsd15,No Pain - Rostam Remix,DJDS,0orEyXhG0KnPEDVgpctNJI,single,56c2Q07YTudOhUKTcl340a,Big Wave More Fire (Remixes),173613.0,45.0,None,['la pop'],pop,la pop
145,4XbZuXFhpVFkAOpmxhxE7K,Run Me Through (King Princess Remix),Perfume Genius,2ueoLVCXQ948OfhVvAy3Nn,single,1JxadQqPZmNNWDdw10Gahe,Run Me Through (King Princess Remix),222127.0,52.0,https://p.scdn.co/mp3-preview/920f299a833fb93e...,"['art pop', 'chamber pop', 'chillwave', 'dance...",indie,art pop
149,4h8TzshCRWhcTQd582ghSo,Just Like Love (Jam City Remix),Perfume Genius,2ueoLVCXQ948OfhVvAy3Nn,single,38ckwzi09abDlt8RP2McK8,Reshaped,208300.0,48.0,https://p.scdn.co/mp3-preview/cce7a430f0a4ddc3...,"['art pop', 'chamber pop', 'chillwave', 'dance...",indie,art pop
198,5iR8v6fTc33LDo2t4w3342,Futile Devices (Doveman Remix),Sufjan Stevens,4MXUO7sVCaFgFjoTI5ox5c,compilation,531pUFZ1BbEhfXDxitF8TU,Call Me by Your Name (Original Motion Picture ...,135443.0,54.0,https://p.scdn.co/mp3-preview/e5360619c7542e32...,"['baroque pop', 'chamber pop', 'freak folk', '...",indie,baroque pop
342,3YDZEi6jetqyeNZdsk8Y5R,Big City Boys - Generationals Remix,BRONCHO,6Lll1MPPak4m4vZKuJB264,single,4n548r94uoKnz0kJbq5lFo,Big City Boys (Generationals Remix),194705.0,41.0,https://p.scdn.co/mp3-preview/586422a7a495a57a...,"['indie garage rock', 'neo-psychedelic', 'norm...",indie,indie garage rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31610,34Oki6j9VX3WmV9WllUJmO,Love Me Like You - J-Vibe Reggae Remix,Little Mix,3e7awlrlDSwF3iM0WBjGMp,single,5duX2rkm2mTEeDZaMmVhi9,Love Me Like You (The Collection),184520.0,38.0,https://p.scdn.co/mp3-preview/c08d53dd1782dce0...,"['dance pop', 'girl group', 'pop', 'post-teen ...",pop,dance pop
31614,3IbpQimWfBXUuSlrCSadc7,Genius - Banx & Ranx Reggae Remix,Sia,5WUlDfRSoLAfcVSX1WnrxN,single,1Xo1BbCbbXo9RkrjFFgGyH,Genius (Banx & Ranx Remixes),194880.0,45.0,https://p.scdn.co/mp3-preview/2d265831fed7bc45...,"['australian dance', 'australian pop', 'dance ...",pop,australian dance
31616,1n0Tvb6HY6EGjp2WHFHtAW,No One - Curtis Lynch Reggae Remix,Alicia Keys,3DiDSECUqqY1AuBP8qtaIa,single,0sZmgYFY0qhrQQeOcmXO6b,No One - EP,236973.0,44.0,https://p.scdn.co/mp3-preview/66fee27d3e13a2dc...,"['hip pop', 'neo soul', 'pop', 'r&b', 'urban c...",pop,hip pop
31617,4XsVJxpSUTIIoDckhW6r5L,With Ur Love (feat. Mike Posner) - Teka & Soul...,Cher Lloyd,4m4SfDVbF5wxrwEjDKgi4k,single,5bgU9eChoaXgc5DMhRSyug,With Ur Love (feat. Mike Posner),228066.0,38.0,https://p.scdn.co/mp3-preview/c7d2095d74de3362...,"['dance pop', 'electropop', 'pop', 'post-teen ...",pop,dance pop


In [ ]:
data.drop_duplicates(['track_name','artist'])

In [3]:
q = '''SELECT * FROM librosa_features'''
librosa_data = run_query(q)
q = '''SELECT * FROM spotify_features'''
spotify_data = run_query(q)
# q = '''SELECT * FROM norm_tracks'''
# norm_data = run_query(q)

In [87]:
norm_data['track_id'].duplicated().value_counts()

False    21369
Name: track_id, dtype: int64

In [15]:
data_not_null = data[data['track_preview_url'].isnull()==False]

In [18]:
spotify_data['id'].duplicated().value_counts()

False    33096
Name: id, dtype: int64

In [20]:
librosa_data['track_id'].duplicated().value_counts()

False    21426
Name: track_id, dtype: int64

In [21]:
data[data['artist'] == 'My Chemical Romance']['track_id']

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
3417,5wQnmLuC1W7ATsArWACrgW,Welcome to the Black Parade,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,311106.0,77.0,https://p.scdn.co/mp3-preview/0c19f3a680ff92f9...,"['emo', 'modern rock', 'pop punk']",rock,emo
3418,7j31rVgGX9Q2blT92VBEA0,Teenagers,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,161920.0,76.0,https://p.scdn.co/mp3-preview/7e780e073cb38e63...,"['emo', 'modern rock', 'pop punk']",rock,emo
3419,7lRlq939cDG4SzWOF4VAnd,I'm Not Okay (I Promise),My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,3DuiGV3J09SUhvp8gqNx8h,Three Cheers for Sweet Revenge,186480.0,73.0,https://p.scdn.co/mp3-preview/b5ad0ef700a456e3...,"['emo', 'modern rock', 'pop punk']",rock,emo
3420,5dTHtzHFPyi8TlTtzoz1J9,Helena,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,3DuiGV3J09SUhvp8gqNx8h,Three Cheers for Sweet Revenge,204733.0,73.0,https://p.scdn.co/mp3-preview/3622f88f8e079672...,"['emo', 'modern rock', 'pop punk']",rock,emo
3421,2d6m2F4I7wCuAKtSsdhh83,Famous Last Words,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,299226.0,70.0,https://p.scdn.co/mp3-preview/67c92af6b1777a1a...,"['emo', 'modern rock', 'pop punk']",rock,emo
3422,4RAOI1etsgbh5NP3T5R8rN,I Don't Love You,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,238680.0,69.0,https://p.scdn.co/mp3-preview/97d0f98973a4de9f...,"['emo', 'modern rock', 'pop punk']",rock,emo
3423,5BB0Jzw60KyfSTyjJqtely,Na Na Na (Na Na Na Na Na Na Na Na Na),My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,2wPnKggTK3QhYAKL7Q0vvr,Danger Days: The True Lives of the Fabulous Ki...,205960.0,69.0,https://p.scdn.co/mp3-preview/67bf2886d9a6f6df...,"['emo', 'modern rock', 'pop punk']",rock,emo
3424,1yKAqZoi8xWGLCf5vajroL,The Ghost of You,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,3DuiGV3J09SUhvp8gqNx8h,Three Cheers for Sweet Revenge,194253.0,67.0,https://p.scdn.co/mp3-preview/e1e50362e2eb4590...,"['emo', 'modern rock', 'pop punk']",rock,emo
3425,0Zh3tKIphLOvQux4dA6PFZ,Mama,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,279026.0,66.0,https://p.scdn.co/mp3-preview/c5628597c5a908b3...,"['emo', 'modern rock', 'pop punk']",rock,emo
3426,0uukw2CgEIApv4IWAjXrBC,Dead!,My Chemical Romance,7FBcuc1gsnv6Y1nwFtNRCb,album,0FZK97MXMm5mUQ8mtudjuK,The Black Parade,195520.0,65.0,https://p.scdn.co/mp3-preview/061be493ff5bc095...,"['emo', 'modern rock', 'pop punk']",rock,emo


In [22]:
data[data['artist'] == 'Miranda Lambert']

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
6041,41FLEFSeRCv19OBn0vLiaH,It All Comes Out in the Wash,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,5Or2XM0Gjy6Y8qlaERqsSn,Wildcard,214786.0,62.0,https://p.scdn.co/mp3-preview/7c494cb7866c9e93...,"['contemporary country', 'country', 'country d...",country,contemporary country
8844,6M3BioqekwOI5erB7vOicl,Bluebird,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,single,40T26sCbX4tAVQjhniDfKK,Bluebird,209760.0,20.0,None,"['contemporary country', 'country', 'country d...",country,contemporary country
8877,02eD9ymfJOJOhM97HYp5R9,The House That Built Me,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,3uczBfJFFSNjSiF8ScC1pA,Revolution,236626.0,65.0,https://p.scdn.co/mp3-preview/fae3074a59e9879c...,"['contemporary country', 'country', 'country d...",country,contemporary country
9004,4uJXqYEqdNF4DijLy7NdgP,Somethin' Bad - [duet with Carrie Underwood],Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,single,10k2pIvPJCgVJzXOG8tbwb,Somethin' Bad,169586.0,2.0,None,"['contemporary country', 'country', 'country d...",country,contemporary country
9631,6s9m5J92By7jii22Q2XtY2,Somethin' Bad - (Duet with Carrie Underwood),Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,4ZnIw6llQFlXcFgjMNtrTw,Platinum,169586.0,62.0,https://p.scdn.co/mp3-preview/88cfc4e205a1f599...,"['contemporary country', 'country', 'country d...",country,contemporary country
10548,3fDsErbmOD3TuaSDuxSXav,Over You,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,1Pv6Znx8kGygusP4fqBD7m,Four The Record,253373.0,56.0,https://p.scdn.co/mp3-preview/bc6c35168cdfc69f...,"['contemporary country', 'country', 'country d...",country,contemporary country
10617,4sl4UWqzHSG7riBKdralwy,Little Red Wagon,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,4ZnIw6llQFlXcFgjMNtrTw,Platinum,204386.0,54.0,https://p.scdn.co/mp3-preview/d1c8ac64da1234b9...,"['contemporary country', 'country', 'country d...",country,contemporary country
10689,0B0y2FImat9j9MJTNRZWfW,Automatic,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,4ZnIw6llQFlXcFgjMNtrTw,Platinum,247826.0,65.0,https://p.scdn.co/mp3-preview/9336313b810a9267...,"['contemporary country', 'country', 'country d...",country,contemporary country
10721,2mXsZsJBThTaJXbNcg5QD4,Mama's Broken Heart,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,1Pv6Znx8kGygusP4fqBD7m,Four The Record,177773.0,65.0,https://p.scdn.co/mp3-preview/acb6a1fe8608f0f9...,"['contemporary country', 'country', 'country d...",country,contemporary country
10799,2yhhcHTfvLC0HzSajGYL0h,Kerosene,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,1NovG9mTUookEBrlHP6QC1,Kerosene,185800.0,57.0,https://p.scdn.co/mp3-preview/4f2921e31e95ec19...,"['contemporary country', 'country', 'country d...",country,contemporary country


In [96]:
test_ids = norm_data[norm_data['artist']=='My Chemical Romance']['track_id'].to_list()

In [97]:
data[data['track_id'].isin(test_ids)]

,track_id,track_name,artist,artist_id,track_album_album_type,track_album_id,track_album_name,track_duration_ms,track_popularity,track_preview_url,subgenres,genre,top_subgenre
10688,5zMRkxgFscsPItPsMy4ItS,The Man I Want to Be,Chris Young,4BYxqVkZyFjtik7crYLg5Q,album,6Aormv05kybauFdXCzUEcH,The Man I Want To Be,207213.0,56.0,https://p.scdn.co/mp3-preview/3b388bde1f4acd42...,"['contemporary country', 'country', 'country r...",country,contemporary country
10689,0B0y2FImat9j9MJTNRZWfW,Automatic,Miranda Lambert,66lH4jAE7pqPlOlzUKbwA0,album,4ZnIw6llQFlXcFgjMNtrTw,Platinum,247826.0,65.0,https://p.scdn.co/mp3-preview/9336313b810a9267...,"['contemporary country', 'country', 'country d...",country,contemporary country
10693,1rGLPyb4LMojT4wpxiZTp8,Sunny and 75,Joe Nichols,4aJTB79uwT4sP0Nb8QGWHc,album,73I0czjxZIGfvYBpf1j98v,Crickets,206946.0,57.0,https://p.scdn.co/mp3-preview/993a1ae65269da94...,"['arkansas country', 'contemporary country', '...",country,arkansas country
10695,3u0iJcNE8fHupAO2OOqGlC,Everybody Wants to Go to Heaven (with The Wail...,Kenny Chesney,3grHWM9bx2E9vwJCdlRv9O,album,4WZiQjmX5wVzQV7Aqk9AVP,Lucky Old Sun,252986.0,46.0,https://p.scdn.co/mp3-preview/ebdd0cec9f5b9b21...,"['contemporary country', 'country', 'country r...",country,contemporary country
10701,5pgmgHNxiq9w9XuthMKzJS,Beer In Mexico,Kenny Chesney,3grHWM9bx2E9vwJCdlRv9O,album,5uMLCobPh02IG3fyH0Dzec,The Road And The Radio,271813.0,56.0,https://p.scdn.co/mp3-preview/00c94fcc117b7181...,"['contemporary country', 'country', 'country r...",country,contemporary country
10702,27aDju7VAODvQYHWrrv6o9,Goodnight Kiss,Randy Houser,56x8mYvS3cyDGAi8N2FxbB,album,4BVzcXLDJ8pRtJttQiU3G6,How Country Feels,206800.0,51.0,https://p.scdn.co/mp3-preview/bb38c2c7a0d41ea7...,"['contemporary country', 'country', 'country r...",country,contemporary country
10708,3QV7NYkrmV0Q0IHdFJw9hO,She Thinks My Tractor's Sexy,Kenny Chesney,3grHWM9bx2E9vwJCdlRv9O,album,67qNNUSHekqSliS7Q1XZj6,Everywhere We Go,248066.0,64.0,https://p.scdn.co/mp3-preview/7c2575ea27b87e71...,"['contemporary country', 'country', 'country r...",country,contemporary country
10715,6YvKQtwW8YgIqRDbb4dFDK,Back When,Tim McGraw,6roFdX1y5BYSbp60OTJWMd,compilation,7pul7RvuYLqryKehizOvsG,35 Biggest Hits,269893.0,44.0,https://p.scdn.co/mp3-preview/cfeb9f913f8826da...,"['contemporary country', 'country', 'country r...",country,contemporary country
10745,0gY2iq0xJPRoIB1PScKSw4,Honey Bee,Blake Shelton,1UTPBmNbXNTittyMJrNkvw,album,1WXCjIDp84rJN6Sa1Um9kJ,Red River Blue (Deluxe Edition),210720.0,63.0,https://p.scdn.co/mp3-preview/77ff81b6f17cef94...,"['contemporary country', 'country', 'country r...",country,contemporary country
10749,6ej1QJ8eIYmhsyTlvgDajy,Where I Come From,Alan Jackson,4mxWe1mtYIYfP040G38yvS,compilation,26w8J2SKCxa80Bwq6G8ctN,34 Number Ones,239240.0,53.0,https://p.scdn.co/mp3-preview/fdc442a81d1a57e3...,"['contemporary country', 'country', 'country r...",country,contemporary country


In [29]:
spotify_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.417,0.687,7,-6.484,1,0.0372,0.0955,0.02780,0.0989,0.627,204.110,6YDf6QV7QfCEo8O2dbWalK,189293.0,4
1,0.493,0.437,7,-9.494,1,0.0326,0.3940,0.00197,0.2250,0.425,78.331,0qaWxVVd3KrP4WY27KWpIe,223573.0,4
2,0.534,0.659,0,-8.279,1,0.0270,0.4870,0.08080,0.0892,0.497,107.052,1yZMv2GMAibgLGsQiowZrt,234160.0,4
3,0.436,0.804,0,-4.907,1,0.0409,0.2090,0.00000,0.1710,0.784,188.061,2eg2gvPXuwZ9FyrPaLgrXi,170840.0,4
4,0.583,0.831,9,-3.417,1,0.0289,0.0502,0.27200,0.0400,0.628,94.009,0wzruvvN7f5wu39aFcjTMw,190613.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33091,0.694,0.577,4,-14.630,1,0.0308,0.0150,0.00836,0.0853,0.845,114.478,0a4agFmqHXxcZl1nho1BxM,275733.0,4
33092,0.606,0.848,11,-4.509,1,0.0386,0.2360,0.00000,0.4300,0.831,109.252,1tsWNtWxi7SVydMm2G1W6Z,180587.0,4
33093,0.571,0.652,2,-11.034,0,0.0499,0.5510,0.00000,0.0598,0.947,117.956,3GGcwG519BTMdvMeFy7meT,235267.0,4
33094,0.719,0.547,10,-13.190,1,0.0362,0.0679,0.00000,0.2290,0.554,121.724,1ip1ULxxvAzaqH3VqKObdu,316000.0,4


In [30]:
librosa_data

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,D,D#,E,F,F#,G,G#,A,A#,B
0,6YDf6QV7QfCEo8O2dbWalK,1756.4558,2046.3342,3311.4383,0.0825,-6.8987,128.4723,-26.9279,12.1849,20.8006,...,0.4522,0.3723,0.5536,0.4106,0.4630,0.4977,0.4397,0.5498,0.3668,0.3736
1,0qaWxVVd3KrP4WY27KWpIe,1911.7354,2148.9760,4108.5335,0.0858,-43.1119,113.6926,-16.0353,21.3349,-7.6568,...,0.4439,0.2756,0.3617,0.2811,0.4238,0.5576,0.3802,0.5572,0.3689,0.4886
2,1yZMv2GMAibgLGsQiowZrt,1502.5131,1782.0399,2969.3070,0.0695,-73.8604,138.2465,-25.7173,24.4074,-0.4938,...,0.2634,0.1774,0.4107,0.5217,0.2529,0.2749,0.2159,0.4378,0.1899,0.1266
3,2eg2gvPXuwZ9FyrPaLgrXi,2430.0319,2420.7924,4922.0702,0.1331,-8.5929,91.2195,-15.3536,17.1535,-0.0333,...,0.4748,0.2990,0.4744,0.3313,0.2653,0.3464,0.2482,0.3924,0.2739,0.4180
4,0wzruvvN7f5wu39aFcjTMw,2729.2951,2424.4074,5364.5676,0.1561,10.4661,73.9084,-32.9083,6.7778,1.7658,...,0.2522,0.2841,0.5090,0.4800,0.6721,0.5047,0.5559,0.3559,0.2284,0.2586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21421,2BEArHIJsI5fxYEpCZv2sF,2433.5812,2432.9820,5060.9527,0.1147,-143.8287,81.7018,-14.9814,17.8765,-3.1500,...,0.3916,0.3325,0.2282,0.2407,0.2435,0.3469,0.2709,0.1673,0.1863,0.3064
21422,4OmDRl8nlE1IkWkwS82nY3,2760.9049,2524.4064,5531.0082,0.1530,-39.9065,71.2764,-20.7539,17.3165,0.3001,...,0.4430,0.3828,0.3960,0.3397,0.3339,0.4543,0.3722,0.3542,0.3191,0.4511
21423,3GGcwG519BTMdvMeFy7meT,2278.6410,2133.8203,4512.6977,0.1280,-104.4452,94.0545,-21.7863,30.7619,-18.3952,...,0.5097,0.3333,0.2867,0.3037,0.2499,0.5309,0.3636,0.2976,0.2825,0.2893
21424,1ip1ULxxvAzaqH3VqKObdu,2772.0318,2824.7279,6263.4621,0.1133,-102.1716,70.0221,7.4154,26.8613,7.7186,...,0.3082,0.2867,0.2303,0.2651,0.2903,0.3908,0.4036,0.4104,0.3861,0.2959


In [4]:
joined = librosa_data.set_index('track_id').join(spotify_data.set_index('id'), how='inner')
joined = joined.reset_index()
joined.rename(columns={'index':'track_id'},inplace=True)
joined

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6YDf6QV7QfCEo8O2dbWalK,1756.4558,2046.3342,3311.4383,0.0825,-6.8987,128.4723,-26.9279,12.1849,20.8006,...,-6.484,1,0.0372,0.0955,0.02780,0.0989,0.627,204.110,189293.0,4
1,0qaWxVVd3KrP4WY27KWpIe,1911.7354,2148.9760,4108.5335,0.0858,-43.1119,113.6926,-16.0353,21.3349,-7.6568,...,-9.494,1,0.0326,0.3940,0.00197,0.2250,0.425,78.331,223573.0,4
2,1yZMv2GMAibgLGsQiowZrt,1502.5131,1782.0399,2969.3070,0.0695,-73.8604,138.2465,-25.7173,24.4074,-0.4938,...,-8.279,1,0.0270,0.4870,0.08080,0.0892,0.497,107.052,234160.0,4
3,2eg2gvPXuwZ9FyrPaLgrXi,2430.0319,2420.7924,4922.0702,0.1331,-8.5929,91.2195,-15.3536,17.1535,-0.0333,...,-4.907,1,0.0409,0.2090,0.00000,0.1710,0.784,188.061,170840.0,4
4,0wzruvvN7f5wu39aFcjTMw,2729.2951,2424.4074,5364.5676,0.1561,10.4661,73.9084,-32.9083,6.7778,1.7658,...,-3.417,1,0.0289,0.0502,0.27200,0.0400,0.628,94.009,190613.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21419,2BEArHIJsI5fxYEpCZv2sF,2433.5812,2432.9820,5060.9527,0.1147,-143.8287,81.7018,-14.9814,17.8765,-3.1500,...,-13.398,0,0.0405,0.1090,0.08220,0.0891,0.967,125.816,212009.0,4
21420,4OmDRl8nlE1IkWkwS82nY3,2760.9049,2524.4064,5531.0082,0.1530,-39.9065,71.2764,-20.7539,17.3165,0.3001,...,-9.337,1,0.0752,0.0094,0.01990,0.1700,0.721,117.128,366200.0,4
21421,3GGcwG519BTMdvMeFy7meT,2278.6410,2133.8203,4512.6977,0.1280,-104.4452,94.0545,-21.7863,30.7619,-18.3952,...,-11.034,0,0.0499,0.5510,0.00000,0.0598,0.947,117.956,235267.0,4
21422,1ip1ULxxvAzaqH3VqKObdu,2772.0318,2824.7279,6263.4621,0.1133,-102.1716,70.0221,7.4154,26.8613,7.7186,...,-13.190,1,0.0362,0.0679,0.00000,0.2290,0.554,121.724,316000.0,4


In [5]:
joined.drop(['key','mode','duration_ms','time_signature'],axis=1,inplace=True)

In [6]:
joined.to_csv('../sharpest-minds/spotify/data/raw_tracks.csv',index=None)

In [57]:
joined.head()

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,B,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,6YDf6QV7QfCEo8O2dbWalK,1756.4558,2046.3342,3311.4383,0.0825,-6.8987,128.4723,-26.9279,12.1849,20.8006,...,0.3736,0.417,0.687,-6.484,0.0372,0.0955,0.02780,0.0989,0.627,204.110
1,0qaWxVVd3KrP4WY27KWpIe,1911.7354,2148.9760,4108.5335,0.0858,-43.1119,113.6926,-16.0353,21.3349,-7.6568,...,0.4886,0.493,0.437,-9.494,0.0326,0.3940,0.00197,0.2250,0.425,78.331
2,1yZMv2GMAibgLGsQiowZrt,1502.5131,1782.0399,2969.3070,0.0695,-73.8604,138.2465,-25.7173,24.4074,-0.4938,...,0.1266,0.534,0.659,-8.279,0.0270,0.4870,0.08080,0.0892,0.497,107.052
3,2eg2gvPXuwZ9FyrPaLgrXi,2430.0319,2420.7924,4922.0702,0.1331,-8.5929,91.2195,-15.3536,17.1535,-0.0333,...,0.4180,0.436,0.804,-4.907,0.0409,0.2090,0.00000,0.1710,0.784,188.061
4,0wzruvvN7f5wu39aFcjTMw,2729.2951,2424.4074,5364.5676,0.1561,10.4661,73.9084,-32.9083,6.7778,1.7658,...,0.2586,0.583,0.831,-3.417,0.0289,0.0502,0.27200,0.0400,0.628,94.009


In [7]:
from sklearn import preprocessing

x = joined.iloc[:,1:].values #returns a numpy array
y = joined.columns[1:]
i = joined.loc[:,'track_id']
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
norm_joined = pd.DataFrame(x_scaled, columns=y)
norm_joined.insert(0,'track_id',i)
norm_joined

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,B,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,6YDf6QV7QfCEo8O2dbWalK,0.379020,0.556578,0.368925,0.241848,0.886696,0.478709,0.316089,0.358038,0.661754,...,0.366767,0.425076,0.686687,0.846992,0.039282,0.095884,0.027884,0.087853,0.636548,0.835961
1,0qaWxVVd3KrP4WY27KWpIe,0.416091,0.587983,0.465407,0.251812,0.846555,0.421094,0.366829,0.423349,0.441553,...,0.486049,0.502548,0.436436,0.787923,0.034424,0.395582,0.001976,0.215925,0.431472,0.320816
2,1yZMv2GMAibgLGsQiowZrt,0.318393,0.475713,0.327513,0.202597,0.812472,0.516812,0.321728,0.445281,0.496979,...,0.110569,0.544343,0.658659,0.811767,0.028511,0.488956,0.081043,0.078001,0.504569,0.438447
3,2eg2gvPXuwZ9FyrPaLgrXi,0.539831,0.671150,0.563879,0.394626,0.884818,0.333487,0.370005,0.393503,0.500543,...,0.412820,0.444444,0.803804,0.877939,0.043189,0.209839,0.000000,0.161081,0.795939,0.770230
4,0wzruvvN7f5wu39aFcjTMw,0.611277,0.672256,0.617439,0.464070,0.905944,0.266004,0.288231,0.319443,0.514464,...,0.247485,0.594292,0.830831,0.907178,0.030517,0.050402,0.272818,0.028032,0.637563,0.385027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21419,2BEArHIJsI5fxYEpCZv2sF,0.540678,0.674880,0.580689,0.339070,0.734916,0.296384,0.371738,0.398664,0.476426,...,0.297065,0.798165,0.540541,0.711311,0.042767,0.109438,0.082447,0.077900,0.981726,0.515297
21420,4OmDRl8nlE1IkWkwS82nY3,0.618824,0.702853,0.637586,0.454710,0.850108,0.255743,0.344849,0.394667,0.503123,...,0.447153,0.716616,0.754755,0.791004,0.079409,0.009438,0.019960,0.160065,0.731980,0.479714
21421,3GGcwG519BTMdvMeFy7meT,0.503687,0.583346,0.514328,0.379227,0.778570,0.344539,0.340040,0.490638,0.358459,...,0.279328,0.582059,0.651652,0.757702,0.052693,0.553213,0.000000,0.048141,0.961421,0.483105
21422,1ip1ULxxvAzaqH3VqKObdu,0.621481,0.794741,0.726243,0.334843,0.781090,0.250854,0.476068,0.462796,0.560527,...,0.286174,0.732926,0.546547,0.715393,0.038226,0.068173,0.000000,0.219988,0.562437,0.498538


In [62]:
norm_joined.to_csv('../sharpest-minds/spotify/data/norm_tracks.csv',index=None,sep=';')

In [89]:
norm_data.columns

Index(['track_id', 'track_name', 'artist', 'spectral_centroid',
       'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2',
       'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10',
       'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17',
       'mfcc18', 'mfcc19', 'mfcc20', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G',
       'G#', 'A', 'A#', 'B', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo'],
      dtype='object')

In [8]:
meta = data[['track_id','track_name','artist','subgenres','genre','top_subgenre']]

In [9]:
norm_genre = meta.set_index('track_id').join(norm_joined.set_index('track_id'), how='inner')
norm_genre.drop(['subgenres','genre','top_subgenre'],axis=1,inplace=True)
norm_genre.reset_index(inplace=True)
norm_genre

,track_id,track_name,artist,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,B,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,0.379020,0.556578,0.368925,0.241848,0.886696,0.478709,0.316089,...,0.366767,0.425076,0.686687,0.846992,0.039282,0.095884,0.027884,0.087853,0.636548,0.835961
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,0.416091,0.587983,0.465407,0.251812,0.846555,0.421094,0.366829,...,0.486049,0.502548,0.436436,0.787923,0.034424,0.395582,0.001976,0.215925,0.431472,0.320816
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,0.318393,0.475713,0.327513,0.202597,0.812472,0.516812,0.321728,...,0.110569,0.544343,0.658659,0.811767,0.028511,0.488956,0.081043,0.078001,0.504569,0.438447
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,0.539831,0.671150,0.563879,0.394626,0.884818,0.333487,0.370005,...,0.412820,0.444444,0.803804,0.877939,0.043189,0.209839,0.000000,0.161081,0.795939,0.770230
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,0.611277,0.672256,0.617439,0.464070,0.905944,0.266004,0.288231,...,0.247485,0.594292,0.830831,0.907178,0.030517,0.050402,0.272818,0.028032,0.637563,0.385027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21364,2BEArHIJsI5fxYEpCZv2sF,Ring My Bell,Anita Ward,0.540678,0.674880,0.580689,0.339070,0.734916,0.296384,0.371738,...,0.297065,0.798165,0.540541,0.711311,0.042767,0.109438,0.082447,0.077900,0.981726,0.515297
21365,4OmDRl8nlE1IkWkwS82nY3,Love Sensation,Loleatta Holloway,0.618824,0.702853,0.637586,0.454710,0.850108,0.255743,0.344849,...,0.447153,0.716616,0.754755,0.791004,0.079409,0.009438,0.019960,0.160065,0.731980,0.479714
21366,3GGcwG519BTMdvMeFy7meT,Lady Marmalade,Patti LaBelle,0.503687,0.583346,0.514328,0.379227,0.778570,0.344539,0.340040,...,0.279328,0.582059,0.651652,0.757702,0.052693,0.553213,0.000000,0.048141,0.961421,0.483105
21367,1ip1ULxxvAzaqH3VqKObdu,Can't Take My Eyes Off of You - Black Box Mix,Gloria Gaynor,0.621481,0.794741,0.726243,0.334843,0.781090,0.250854,0.476068,...,0.286174,0.732926,0.546547,0.715393,0.038226,0.068173,0.000000,0.219988,0.562437,0.498538


In [100]:
test_2 = data[data['artist']=='My Chemical Romance']['track_id'].to_list()
norm_genre[norm_genre['track_id'].isin(test_2)]

,track_id,track_name,artist,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,B,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
2535,5wQnmLuC1W7ATsArWACrgW,Welcome to the Black Parade,My Chemical Romance,0.620278,0.718456,0.661423,0.404891,0.933056,0.256874,0.378641,...,0.365522,0.221203,0.904905,0.893716,0.079409,0.000301,0.000110,0.212878,0.239594,0.397072
2536,7j31rVgGX9Q2blT92VBEA0,Teenagers,My Chemical Romance,0.650403,0.693875,0.664856,0.500604,0.946577,0.240531,0.354646,...,0.462089,0.471967,0.856857,0.914125,0.066737,0.050803,0.000000,0.174284,0.869036,0.457266
2537,7lRlq939cDG4SzWOF4VAnd,I'm Not Okay (I Promise),My Chemical Romance,0.603239,0.691645,0.631072,0.433575,0.949706,0.288825,0.363683,...,0.452443,0.214067,0.939940,0.906982,0.129884,0.006024,0.000000,0.260613,0.258883,0.736077
2538,5dTHtzHFPyi8TlTtzoz1J9,Helena,My Chemical Romance,0.549047,0.612660,0.563508,0.402778,0.934655,0.343939,0.304209,...,0.439581,0.362895,0.959960,0.905805,0.109820,0.014257,0.000000,0.199675,0.087310,0.515727
2539,2d6m2F4I7wCuAKtSsdhh83,Famous Last Words,My Chemical Romance,0.607858,0.681749,0.627359,0.429348,0.912651,0.262055,0.371091,...,0.361166,0.476045,0.882883,0.902037,0.111932,0.000301,0.000150,0.114361,0.313706,0.500700
2540,4RAOI1etsgbh5NP3T5R8rN,I Don't Love You,My Chemical Romance,0.545181,0.614528,0.550957,0.414251,0.920341,0.313899,0.321843,...,0.395291,0.294597,0.795796,0.891420,0.052165,0.008434,0.000030,0.192566,0.335025,0.695583
2541,5BB0Jzw60KyfSTyjJqtely,Na Na Na (Na Na Na Na Na Na Na Na Na),My Chemical Romance,0.721367,0.750735,0.729925,0.565519,0.959479,0.195387,0.402633,...,0.434291,0.418960,0.985986,0.924742,0.061563,0.000301,0.001484,0.364209,0.658883,0.679799
2542,1yKAqZoi8xWGLCf5vajroL,The Ghost of You,My Chemical Romance,0.610630,0.708070,0.642574,0.429046,0.962674,0.280583,0.375996,...,0.378384,0.205912,0.885886,0.899564,0.086167,0.028715,0.000000,0.640463,0.194924,0.597067
2543,0Zh3tKIphLOvQux4dA6PFZ,Mama,My Chemical Romance,0.668176,0.725322,0.695244,0.471316,0.947619,0.233324,0.386390,...,0.489679,0.240571,0.798799,0.872836,0.099155,0.013655,0.000000,0.083689,0.307614,0.855788
2544,0uukw2CgEIApv4IWAjXrBC,Dead!,My Chemical Romance,0.613330,0.713863,0.654578,0.420592,0.951931,0.280945,0.378649,...,0.475158,0.490316,0.972973,0.936065,0.057233,0.003715,0.000883,0.317489,0.367513,0.406824


In [10]:
norm_genre.to_csv('../sharpest-minds/spotify/data/norm_tracks.csv',index=None,sep=';')

In [11]:
fv = 

q = '''SELECT a.*, b.*
        FROM librosa_features a 
        JOIN spotify_features b ON a.track_id = b.id;'''

database = run_query(q)
database.drop(['id','duration_ms','time_signature','mode','key'],axis=1, inplace=True)
database
# i = len(database)
# fv = not_in_db_df.drop(['track_name','artist','genre'],axis=1)

# #append feature vector to bottom of the db
# database = pd.concat([database.iloc[:,1:],fv.iloc[:,1:]],ignore_index=True)

# # #apply a lambda function that does min-max normalization on the db
# database = database.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

# #overwrite features vector df
# not_in_db_df.iloc[:,3:-1] = database.iloc[i:,:].values
# return not_in_db_df

,track_id,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,...,B,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3VB5e3mQF5F0qzX5UD7WIx,1624.3276,1889.2187,3147.8226,0.0699,-242.2812,111.5955,17.0803,35.6525,7.3912,...,0.2783,0.522,0.095,-13.483,0.0382,0.9090,0.00000,0.1110,0.173,110.992
1,6od2yPHW6FK8rS3shyKCrQ,2184.9372,2317.4284,4544.7225,0.1045,-24.7281,101.4210,-17.3765,22.3344,8.1569,...,0.3541,0.583,0.886,-4.027,0.0344,0.1080,0.00001,0.1090,0.853,165.016
2,6ZcuwgTbYQ7xz5VN9tBhCw,2119.7028,2438.0190,4659.1802,0.0828,-13.6440,103.6547,2.6560,21.4083,3.8341,...,0.3151,0.473,0.609,-6.870,0.0270,0.2230,0.00000,0.2210,0.523,179.524
3,5MIrEL3UsDmD7RGezOi4AP,2080.9541,2234.2348,4289.7824,0.1031,4.3456,114.6627,-15.8210,13.5587,-3.2852,...,0.3949,0.534,0.663,-5.480,0.0286,0.1280,0.00004,0.4310,0.573,141.525
4,2MOzW8EKpzTSoftk1M1sxN,1700.3852,1999.2797,3327.1632,0.0841,-78.9324,128.8103,-12.0213,19.6843,5.2876,...,0.3088,0.536,0.489,-7.669,0.0319,0.7400,0.00000,0.0948,0.364,104.598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21419,7Geo3JaNL3jkAu1eOPsUF4,2189.4839,2495.4613,4573.8473,0.0928,-102.8407,89.7698,-1.2375,17.2021,12.0410,...,0.3873,0.806,0.840,-6.941,0.0655,0.0033,0.81600,0.0874,0.430,125.495
21420,2LdS0xk7dXvuj3qVUvu042,2070.2525,2225.0579,4235.8911,0.0958,-35.2966,99.0703,-23.6401,24.5888,13.3017,...,0.5631,0.664,0.840,-7.361,0.0534,0.0337,0.77100,0.1730,0.760,154.948
21421,4AMhabRXfjtWNhcwUpNQDv,2873.5358,2527.3347,5683.2990,0.1715,-68.6750,66.0367,-6.9828,25.8186,-4.4582,...,0.3157,0.393,0.405,-11.008,0.0357,0.6560,0.08930,0.1630,0.161,132.899
21422,4OmDRl8nlE1IkWkwS82nY3,2760.9049,2524.4064,5531.0082,0.1530,-39.9065,71.2764,-20.7539,17.3165,0.3001,...,0.4511,0.703,0.755,-9.337,0.0752,0.0094,0.01990,0.1700,0.721,117.128
